In [1]:
import os
import cv2
import numpy as np
from skimage.filters import threshold_niblack, threshold_sauvola

In [2]:
image_folder = "Training/Images"
mask_folder = "Training/Masks"

In [3]:
def sensitivity(pred, gt):
    pred = pred.astype(np.uint8)
    gt = gt.astype(np.uint8)
    TP = np.sum((pred == 1) & (gt == 1))
    FN = np.sum((pred == 0) & (gt == 1))
    if (TP + FN) == 0:
        return 0
    return TP / (TP + FN)

In [ ]:
windows = [15, 25, 35]
niblack_k_values = [-0.2, 0.0, 0.2]
savoula_k_values = [0.1, 0.2, 0.3]
best_niblack_score = 0
best_niblack_params = None
best_sauvola_score = 0
best_sauvola_params = None

In [15]:

for w in windows:
    for k in niblack_k_values:
        sens_list = []
        for image_name in os.listdir(image_folder):
            if "HRF" not in image_name:
                continue
            image_path = os.path.join(image_folder, image_name)
            image = cv2.imread(image_path)
            if image is None:
                continue
            image = image[:, :, 1]
            name = os.path.splitext(image_name)[0]
            mask_path = os.path.join(mask_folder, name + ".tif")
            if not os.path.exists(mask_path):
                continue
            mask = cv2.imread(mask_path, 0)
            mask = mask > 0
            thresh = threshold_niblack(image, window_size=w, k=k)
            binary = image > thresh
            sens = sensitivity(binary, mask)
            sens_list.append(sens)
        avg_sens = np.mean(sens_list)
        print(f"Niblack -> window={w}, k={k}, sensitivity={avg_sens:.4f}")
        if avg_sens > best_niblack_score:
            best_niblack_score = avg_sens
            best_niblack_params = (w, k)

Niblack -> window=15, k=-0.2, sensitivity=0.1343
Niblack -> window=15, k=0.0, sensitivity=0.1986
Niblack -> window=15, k=0.2, sensitivity=0.2890
Niblack -> window=25, k=-0.2, sensitivity=0.0784
Niblack -> window=25, k=0.0, sensitivity=0.1217
Niblack -> window=25, k=0.2, sensitivity=0.1828
Niblack -> window=35, k=-0.2, sensitivity=0.0573
Niblack -> window=35, k=0.0, sensitivity=0.0899
Niblack -> window=35, k=0.2, sensitivity=0.1354


In [13]:
for w in windows:
    for k in savoula_k_values:
        sens_list = []
        for image_name in os.listdir(image_folder):
            if "HRF" not in image_name:
                continue
            image = cv2.imread(os.path.join(image_folder, image_name))
            image = image[:, :, 1]
            name = os.path.splitext(image_name)[0]
            mask = cv2.imread(os.path.join(mask_folder, name + ".tif"), 0)
            mask = mask > 0
            thresh = threshold_sauvola(image, window_size=w, k=k)
            binary = image > thresh
            sens = sensitivity(binary, mask)
            sens_list.append(sens)
        avg = np.mean(sens_list)
        print("Window:", w, "k:", k, "Sensitivity:", avg)
        if avg > best_sauvola_score:
            best_sauvola_score = avg
            best_sauvola_params = (w, k)

Window: 15 k: 0.1 Sensitivity: 0.8247965114829963
Window: 15 k: 0.2 Sensitivity: 0.9874006999411234
Window: 15 k: 0.3 Sensitivity: 0.9992874755604082
Window: 25 k: 0.1 Sensitivity: 0.5856023582357534
Window: 25 k: 0.2 Sensitivity: 0.9178787980402827
Window: 25 k: 0.3 Sensitivity: 0.99089832514085
Window: 35 k: 0.1 Sensitivity: 0.4488111211920114
Window: 35 k: 0.2 Sensitivity: 0.807987445517458
Window: 35 k: 0.3 Sensitivity: 0.9637232613330468


In [16]:
print("Niblack: ")
print("Best Niblack Parameters:", best_niblack_params)
print("Best Niblack Sensitivity:", best_niblack_score)
print("\nSauvola: ")
print("Best Sauvola Window:", best_sauvola_params)
print("Best Sauvola Sensitivity:", best_sauvola_score)


Niblack: 
Best Niblack Parameters: (15, 0.2)
Best Niblack Sensitivity: 0.2889828589009251

Sauvola: 
Best Sauvola Window: (15, 0.3)
Best Sauvola Sensitivity: 0.9992874755604082


Niblack Observation:
 - Smaller window sizes performed better.
 - Increasing window size reduced sensitivity.
 - Overall sensitivity remained relatively low.

Sauvola Observation:
 - Sauvola consistently outperformed Niblack.
 - Higher k values improved sensitivity.
 - Smaller window sizes gave better vessel detection.

Sauvola achieved significantly higher sensitivity compared to Niblack.